In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datasets import load_dataset
import torch
from transformers import AutoTokenizer, AutoModel
from datasets import Dataset
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
import os

device = "cuda" if torch.cuda.is_available() else "cpu"


def run_experiments(n_runs, X, y, study):
    accuracies = []
    random_seeds = np.random.randint(0, 10000, size=n_runs)
    for seed in tqdm(random_seeds, desc=f"Running {study} Experiments"):
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=seed
        )
        clf = RandomForestClassifier(n_estimators=100, random_state=seed)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)
    print(f"{study} Mean Accuracy: {mean_accuracy:.4f} ± {std_accuracy:.4f}")


def generate_finetuned_embeddings(gatortron_pretrained_data):
    if not os.path.exists(FINE_TUNED_MODEL):
        dataset = gatortron_pretrained_data
        model = AutoModel.from_pretrained(FINE_TUNED_MODEL)
        tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL)
        model.eval()
        model.to(device)

        def tokenize_function(examples):
            return tokenizer(
                examples["text"], padding="max_length", truncation=True, max_length=512
            )

        tokenized_dataset = dataset.map(tokenize_function, batched=True)
        tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask"])
        dataloader = DataLoader(tokenized_dataset, batch_size=16)
        embeddings = []
        with torch.no_grad():
            for batch in tqdm(dataloader, desc="Generating Embeddings"):
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                outputs = model(input_ids, attention_mask=attention_mask)
                batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
                embeddings.extend(batch_embeddings)
        dataset = dataset.to_pandas()
        dataset["embeddings"] = list(map(list, embeddings))
        dataset.to_parquet(EMBEDDINGS_SAVE_PATH)
    gatortron_finetuned_data = pd.read_parquet(EMBEDDINGS_SAVE_PATH)
    return gatortron_finetuned_data

## GatorTron

In [ ]:
EMBEDDINGS_SAVE_PATH = "ft_all_data_with_embeddings.parquet"
PRE_TRAINED_MODEL = "UFNLP/gatortron-medium"
FINE_TUNED_MODEL = "./gatortron-base-tcga"

gatortron_pretrained_data = load_dataset("Lab-Rasool/TCGA", "clinical", split="train")
gatortron_finetuned_data = generate_finetuned_embeddings(gatortron_pretrained_data)
gatortron_pretrained_data = gatortron_pretrained_data.to_pandas()

X_gatortron_pretrained = np.array(
    [np.frombuffer(e, dtype=np.float32) for e in gatortron_pretrained_data["embedding"]]
)
y_gatortron_pretrained = gatortron_pretrained_data["project_id"].values
X_gatortron_finetuned = np.array(
    [np.frombuffer(e, dtype=np.float32) for e in gatortron_finetuned_data["embeddings"]]
)
y_gatortron_finetuned = gatortron_finetuned_data["project_id"].values

# ------------------- Train a classifier on the embeddings -------------------
# Run the experiment multiple times with different random seeds
run_experiments(
    n_runs=10,
    study="[UFNLP/gatortron-medium] Pre-trained",
    X=X_gatortron_pretrained,
    y=y_gatortron_pretrained,
)
run_experiments(
    n_runs=10,
    study="[UFNLP/gatortron-medium] Fine-tuned",
    X=X_gatortron_finetuned,
    y=y_gatortron_finetuned,
)

# ------------------- Visualize the embeddings -------------------
tsne = TSNE(n_components=2, random_state=42)
X_reduced = tsne.fit_transform(X_gatortron_pretrained)
unique_projects = np.unique(y_gatortron_pretrained)
palette = sns.color_palette("hsv", len(unique_projects))
project_to_color = {project: palette[i] for i, project in enumerate(unique_projects)}
colors = [project_to_color[project] for project in y_gatortron_pretrained]
plt.figure(figsize=(18, 8), dpi=600)
plt.subplot(1, 2, 1)
sns.scatterplot(
    x=X_reduced[:, 0],
    y=X_reduced[:, 1],
    hue=y_gatortron_pretrained,
    legend=None,
    palette=palette,
)
plt.title("t-SNE of Pre-trained Embeddings", fontsize=18)
plt.xlabel("t-SNE Component 1", fontsize=18)
plt.ylabel("t-SNE Component 2", fontsize=18)

tsne = TSNE(n_components=2, random_state=42)
X_reduced_ft = tsne.fit_transform(X_gatortron_finetuned)
colors_ft = [project_to_color[project] for project in y_gatortron_finetuned]
plt.subplot(1, 2, 2)
sns.scatterplot(
    x=X_reduced_ft[:, 0],
    y=X_reduced_ft[:, 1],
    hue=y_gatortron_finetuned,
    legend="full",
    palette=palette,
)
plt.title("t-SNE of Fine-tuned Embeddings", fontsize=18)
plt.xlabel("t-SNE Component 1", fontsize=18)
plt.ylabel("t-SNE Component 2", fontsize=18)
plt.legend(loc="upper right", bbox_to_anchor=(1.2, 1.0), title="Project ID")
plt.tight_layout()
plt.show()

# # ------------------- Generate or Load embeddings -------------------
# if not os.path.exists(EMBEDDINGS_SAVE_PATH):
#     dataset = Dataset.from_pandas(all_data)
#     model = AutoModel.from_pretrained(FINE_TUNED_MODEL)
#     tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL)
#     model.eval()
#     model.to(device)
#     def tokenize_function(examples):
#         return tokenizer(
#             examples["text"], padding="max_length", truncation=True, max_length=512
#         )
#     tokenized_dataset = dataset.map(tokenize_function, batched=True)
#     tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask"])
#     dataloader = DataLoader(tokenized_dataset, batch_size=16)
#     embeddings = []
#     with torch.no_grad():
#         for batch in tqdm(dataloader, desc="Generating Embeddings"):
#             input_ids = batch["input_ids"].to(device)
#             attention_mask = batch["attention_mask"].to(device)
#             outputs = model(input_ids, attention_mask=attention_mask)
#             batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
#             embeddings.extend(batch_embeddings)
#     all_data["embeddings"] = list(map(list, embeddings))
#     all_data.to_parquet(EMBEDDINGS_SAVE_PATH)
# else:
#     all_data = pd.read_parquet(EMBEDDINGS_SAVE_PATH)
#     X = np.array([np.frombuffer(e, dtype=np.float32) for e in all_data["embeddings"]])
#     y = all_data["project_id"].values

## BERT

In [ ]:
ALL_DATA_PATH = "bert_all_data_with_embeddings.parquet"
EMBEDDINGS_SAVE_PATH = "ft_bert_all_data_with_embeddings.parquet"
PRE_TRAINED_MODEL = "bert-base-uncased"
FINE_TUNED_MODEL = "./bert-base-uncased-tcga"

bert_pretrained_data = pd.read_parquet(ALL_DATA_PATH)
bert_finetuned_data = generate_finetuned_embeddings(bert_pretrained_data)

X_bert_pretrained = np.array(
    [np.frombuffer(e, dtype=np.float32) for e in bert_pretrained_data["embedding"]]
)
y_bert_pretrained = bert_pretrained_data["project_id"].values
X_bert_finetuned = np.array(
    [np.frombuffer(e, dtype=np.float32) for e in bert_finetuned_data["embeddings"]]
)
y_bert_finetuned = bert_finetuned_data["project_id"].values

# ------------------- Train a classifier on the embeddings -------------------
# Run the experiment multiple times with different random seeds
run_experiments(
    n_runs=10,
    study="[bert-base-uncased] Pre-trained",
    X=X_bert_pretrained,
    y=y_bert_pretrained,
)
run_experiments(
    n_runs=10,
    study="[bert-base-uncased] Fine-tuned",
    X=X_bert_finetuned,
    y=y_bert_finetuned,
)

# ------------------- Visualize the embeddings -------------------
tsne = TSNE(n_components=2, random_state=42)
X_reduced = tsne.fit_transform(X_bert_pretrained)
unique_projects = np.unique(y_bert_pretrained)
palette = sns.color_palette("hsv", len(unique_projects))
project_to_color = {project: palette[i] for i, project in enumerate(unique_projects)}
colors = [project_to_color[project] for project in y_bert_pretrained]
plt.figure(figsize=(18, 8), dpi=600)
plt.subplot(1, 2, 1)
sns.scatterplot(
    x=X_reduced[:, 0],
    y=X_reduced[:, 1],
    hue=y_bert_pretrained,
    legend=None,
    palette=palette,
)
plt.title("t-SNE of Pre-trained Embeddings", fontsize=18)
plt.xlabel("t-SNE Component 1", fontsize=18)
plt.ylabel("t-SNE Component 2", fontsize=18)

tsne = TSNE(n_components=2, random_state=42)
X_reduced_ft = tsne.fit_transform(X_bert_finetuned)
colors_ft = [project_to_color[project] for project in y_gatortron_finetuned]
plt.subplot(1, 2, 2)
sns.scatterplot(
    x=X_reduced_ft[:, 0],
    y=X_reduced_ft[:, 1],
    hue=y_bert_finetuned,
    legend="full",
    palette=palette,
)
plt.title("t-SNE of Fine-tuned Embeddings", fontsize=18)
plt.xlabel("t-SNE Component 1", fontsize=18)
plt.ylabel("t-SNE Component 2", fontsize=18)
plt.legend(loc="upper right", bbox_to_anchor=(1.2, 1.0), title="Project ID")
plt.tight_layout()
plt.show()

## Retreival Benchmark

In [1]:
import numpy as np
import faiss
import random
import torch
from datasets import load_dataset
import pandas as pd
import os
from transformers import AutoTokenizer, AutoModel
from datasets import Dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm


# Constants for Gatortron
GATOR_EMBEDDINGS_SAVE_PATH = "ft_all_data_with_embeddings.parquet"
GATOR_PRE_TRAINED_MODEL = "UFNLP/gatortron-medium"
GATOR_FINE_TUNED_MODEL = "./gatortron-base-tcga"

# Constants for BERT
ALL_DATA_PATH = "bert_all_data_with_embeddings.parquet"
BERT_EMBEDDINGS_SAVE_PATH = "ft_bert_all_data_with_embeddings.parquet"
BERT_PRE_TRAINED_MODEL = "bert-base-uncased"
BERT_FINE_TUNED_MODEL = "./bert-base-uncased-tcga"

device = "cuda" if torch.cuda.is_available() else "cpu"


# Function to generate fine-tuned embeddings
def generate_finetuned_embeddings(
    pretrained_data, pretrained_model, fine_tuned_model, embeddings_save_path
):
    if not os.path.exists(embeddings_save_path):
        dataset = Dataset.from_pandas(pretrained_data)
        model = AutoModel.from_pretrained(fine_tuned_model)
        tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
        model.eval()
        model.to(device)

        def tokenize_function(examples):
            return tokenizer(
                examples["text"], padding="max_length", truncation=True, max_length=512
            )

        tokenized_dataset = dataset.map(tokenize_function, batched=True)
        tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask"])
        dataloader = DataLoader(tokenized_dataset, batch_size=16)
        embeddings = []
        with torch.no_grad():
            for batch in tqdm(dataloader, desc="Generating Embeddings"):
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                outputs = model(input_ids, attention_mask=attention_mask)
                batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
                embeddings.extend(batch_embeddings)
        dataset = dataset.to_pandas()
        dataset["embeddings"] = list(map(list, embeddings))
        dataset.to_parquet(embeddings_save_path)
    finetuned_data = pd.read_parquet(embeddings_save_path)
    return finetuned_data


# Load the Gatortron dataset
gatortron_pretrained_data = load_dataset("Lab-Rasool/TCGA", "clinical", split="train")
gatortron_finetuned_data = generate_finetuned_embeddings(
    gatortron_pretrained_data.to_pandas(),
    GATOR_PRE_TRAINED_MODEL,
    GATOR_FINE_TUNED_MODEL,
    GATOR_EMBEDDINGS_SAVE_PATH,
)
gatortron_pretrained_data = gatortron_pretrained_data.to_pandas()

# Convert embeddings from the Gatortron dataset
X_gatortron_pretrained = np.array(
    [np.frombuffer(e, dtype=np.float32) for e in gatortron_pretrained_data["embedding"]]
)
y_gatortron_pretrained = gatortron_pretrained_data["project_id"].values
X_gatortron_finetuned = np.array(
    [np.array(e, dtype=np.float32) for e in gatortron_finetuned_data["embeddings"]]
)
y_gatortron_finetuned = gatortron_finetuned_data["project_id"].values

# Load the BERT dataset
bert_pretrained_data = pd.read_parquet(ALL_DATA_PATH)
bert_finetuned_data = generate_finetuned_embeddings(
    bert_pretrained_data,
    BERT_PRE_TRAINED_MODEL,
    BERT_FINE_TUNED_MODEL,
    BERT_EMBEDDINGS_SAVE_PATH,
)

# Convert embeddings from the BERT dataset
X_bert_pretrained = np.array(
    [np.frombuffer(e, dtype=np.float32) for e in bert_pretrained_data["embedding"]]
)
y_bert_pretrained = bert_pretrained_data["project_id"].values
X_bert_finetuned = np.array(
    [np.array(e, dtype=np.float32) for e in bert_finetuned_data["embeddings"]]
)
y_bert_finetuned = bert_finetuned_data["project_id"].values


# Function to perform similarity search and evaluate matching project_ids
def run_benchmark(X_embeddings, y_labels, num_trials=10):
    # Build the FAISS index
    dimension = X_embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)  # Using L2 distance
    index.add(X_embeddings)

    total_matching_count = 0

    for _ in range(num_trials):
        # Pick a random patient embedding for evaluation
        random_index = random.randint(0, len(X_embeddings) - 1)
        query_embedding = X_embeddings[random_index]
        query_project_id = y_labels[random_index]

        # Retrieve the next 10 closest patients
        distances, indices = index.search(
            query_embedding.reshape(1, -1), k=11
        )  # k=11 to include the query patient itself

        # Remove the first index (the query patient itself)
        indices = indices[0][1:]
        distances = distances[0][1:]

        # Check for matching "project_id"
        matching_count = 0
        for idx in indices:
            if y_labels[int(idx)] == query_project_id:  # Convert idx to int
                matching_count += 1

        total_matching_count += matching_count

    average_matching_count = total_matching_count / num_trials
    average_matching_count = (
        average_matching_count / 10
    )  # Divide by 10 to get the percentage
    return average_matching_count


def multi_run_benchmark(X_embeddings, y_labels, num_trials=10, num_runs=10):
    results = []
    for _ in tqdm(range(num_runs), leave=False):
        results.append(run_benchmark(X_embeddings, y_labels, num_trials))
    return results


# Run benchmarks
num_trials = 100
num_runs = 100
results_gatortron_pretrained = multi_run_benchmark(
    X_gatortron_pretrained,
    y_gatortron_pretrained,
    num_trials=num_trials,
    num_runs=num_runs,
)
print(
    f"Gatortron pretrained embeddings: {np.mean(results_gatortron_pretrained):.4f} ± {np.std(results_gatortron_pretrained):.4f}"
)
results_gatortron_finetuned = multi_run_benchmark(
    X_gatortron_finetuned,
    y_gatortron_finetuned,
    num_trials=num_trials,
    num_runs=num_runs,
)
print(
    f"Gatortron fine-tuned embeddings: {np.mean(results_gatortron_finetuned):.4f} ± {np.std(results_gatortron_finetuned):.4f}"
)
results_bert_pretrained = multi_run_benchmark(
    X_bert_pretrained,
    y_bert_pretrained,
    num_trials=num_trials,
    num_runs=num_runs,
)
print(
    f"BERT pretrained embeddings: {np.mean(results_bert_pretrained):.4f} ± {np.std(results_bert_pretrained):.4f}"
)
results_bert_finetuned = multi_run_benchmark(
    X_bert_finetuned,
    y_bert_finetuned,
    num_trials=num_trials,
    num_runs=num_runs,
)
print(
    f"BERT fine-tuned embeddings: {np.mean(results_bert_finetuned):.4f} ± {np.std(results_bert_finetuned):.4f}"
)

  0%|          | 0/100 [00:00<?, ?it/s]

Gatortron pretrained embeddings: 0.6937 ± 0.0336


  0%|          | 0/100 [00:00<?, ?it/s]

Gatortron fine-tuned embeddings: 0.9433 ± 0.0157


  0%|          | 0/100 [00:00<?, ?it/s]

BERT pretrained embeddings: 0.6890 ± 0.0333


  0%|          | 0/100 [00:00<?, ?it/s]

BERT fine-tuned embeddings: 0.8596 ± 0.0286


## Fine-tuning

In [ ]:
# fine-tune the gatortron-base model on the TCGA dataset
from transformers import (
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    AutoModelForSequenceClassification,
)
from datasets import Dataset
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
)
from transformers import BitsAndBytesConfig
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
import torch


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average="weighted"
    )
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


def prepare_data_and_tokenize(all_data, tokenizer):
    # Ensure 'project_id' is used as a label and it is in an appropriate format
    unique_labels = all_data["project_id"].unique().tolist()
    label_dict = {label: i for i, label in enumerate(unique_labels)}
    all_data["labels"] = all_data["project_id"].map(label_dict)

    # Convert DataFrame to a Dataset object
    dataset = Dataset.from_pandas(all_data[["text", "labels"]])

    # Tokenization function to prepare input data
    def tokenize_function(examples):
        model_inputs = tokenizer(
            examples["text"], padding="max_length", truncation=True, max_length=512
        )
        model_inputs["labels"] = examples[
            "labels"
        ]  # Ensure labels are included for the model to compute loss
        return model_inputs

    # Apply tokenization
    tokenized_datasets = dataset.map(tokenize_function, batched=True)

    # Split the dataset
    tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.2)
    return tokenized_datasets["train"], tokenized_datasets["test"]


unique_labels = all_data["project_id"].unique().tolist()

# Load tokenizer and model
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained("UFNLP/gatortron-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "UFNLP/gatortron-base",
    num_labels=len(unique_labels),
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
)
config = LoraConfig(
    r=4,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["key", "query", "value"],
    bias="none",
    task_type="CAUSAL_LM",
)
lora_model = get_peft_model(model, config)

# Prepare and tokenize data
train_dataset, test_dataset = prepare_data_and_tokenize(all_data, tokenizer)

# Updated training arguments
training_args = TrainingArguments(
    output_dir="./results",  # Store all outputs and model checkpoints here.
    evaluation_strategy="epoch",  # Evaluation is done at the end of each epoch.
    save_strategy="epoch",  # Save checkpoints at the end of each epoch to align with evaluations.
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",  # Directory for storing logs
    logging_steps=10,  # Log training info every 10 steps.
    load_best_model_at_end=True,  # Load the best model found during training at the end.
    metric_for_best_model="accuracy",  # Use accuracy to determine the best model.
    fp16=True,  # Use mixed precision training.
)


# Initialize the Trainer
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,  # Function to compute metrics during evaluation
)

# Train the model
train_result = trainer.train()

# Print training results
print("Training completed. Here are the stats:")
print(train_result)

# Evaluate the model
predictions = trainer.predict(test_dataset)
y_pred = np.argmax(predictions.predictions, axis=-1)
print(classification_report(test_dataset["labels"], y_pred))

MODEL_SAVE_FOLDER_NAME = "gatortron-base-tcga"
trainer.model.save_pretrained(MODEL_SAVE_FOLDER_NAME)
trainer.save_model(MODEL_SAVE_FOLDER_NAME)
trainer.model.config.save_pretrained(MODEL_SAVE_FOLDER_NAME)

---

In [ ]:
# # fine-tune the bert-base-uncased model on the TCGA dataset
# from transformers import (
#     AutoTokenizer,
#     TrainingArguments,
#     Trainer,
#     AutoModelForSequenceClassification,
# )
# from datasets import Dataset
# import pandas as pd
# import numpy as np
# from sklearn.metrics import (
#     accuracy_score,
#     precision_recall_fscore_support,
#     classification_report,
# )
# from transformers import BitsAndBytesConfig
# from peft import LoraConfig, get_peft_model
# import torch


# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     precision, recall, f1, _ = precision_recall_fscore_support(
#         labels, predictions, average="weighted"
#     )
#     acc = accuracy_score(labels, predictions)
#     return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


# def prepare_data_and_tokenize(all_data, tokenizer):
#     # Ensure 'project_id' is used as a label and it is in an appropriate format
#     unique_labels = all_data["project_id"].unique().tolist()
#     label_dict = {label: i for i, label in enumerate(unique_labels)}
#     all_data["labels"] = all_data["project_id"].map(label_dict)

#     # Convert DataFrame to a Dataset object
#     dataset = Dataset.from_pandas(all_data[["text", "labels"]])

#     # Tokenization function to prepare input data
#     def tokenize_function(examples):
#         model_inputs = tokenizer(
#             examples["text"], padding="max_length", truncation=True, max_length=512
#         )
#         model_inputs["labels"] = examples[
#             "labels"
#         ]  # Ensure labels are included for the model to compute loss
#         return model_inputs

#     # Apply tokenization
#     tokenized_datasets = dataset.map(tokenize_function, batched=True)

#     # Split the dataset
#     tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.2)
#     return tokenized_datasets["train"], tokenized_datasets["test"]


# unique_labels = all_data["project_id"].unique().tolist()

# # Load tokenizer and model
# quantization_config = BitsAndBytesConfig(load_in_8bit=True)
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# model = AutoModelForSequenceClassification.from_pretrained(
#     "bert-base-uncased",
#     num_labels=len(unique_labels),
#     torch_dtype=torch.float16,
#     quantization_config=quantization_config,
# )
# config = LoraConfig(
#     r=4,
#     lora_alpha=16,
#     lora_dropout=0.1,
#     target_modules=["key", "query", "value"],
#     bias="none",
#     task_type="CAUSAL_LM",
# )
# lora_model = get_peft_model(model, config)

# # Prepare and tokenize data
# train_dataset, test_dataset = prepare_data_and_tokenize(all_data, tokenizer)

# # Updated training arguments
# training_args = TrainingArguments(
#     output_dir="./results",  # Store all outputs and model checkpoints here.
#     evaluation_strategy="epoch",  # Evaluation is done at the end of each epoch.
#     save_strategy="epoch",  # Save checkpoints at the end of each epoch to align with evaluations.
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=16,
#     num_train_epochs=10,
#     weight_decay=0.01,
#     logging_dir="./logs",  # Directory for storing logs
#     logging_steps=10,  # Log training info every 10 steps.
#     load_best_model_at_end=True,  # Load the best model found during training at the end.
#     metric_for_best_model="accuracy",  # Use accuracy to determine the best model.
#     fp16=True,  # Use mixed precision training.
# )


# # Initialize the Trainer
# trainer = Trainer(
#     model=lora_model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     compute_metrics=compute_metrics,  # Function to compute metrics during evaluation
# )

# # Train the model
# train_result = trainer.train()

# # Print training results
# print("Training completed. Here are the stats:")
# print(train_result)

# # Evaluate the model
# predictions = trainer.predict(test_dataset)
# y_pred = np.argmax(predictions.predictions, axis=-1)
# print(classification_report(test_dataset["labels"], y_pred))

# MODEL_SAVE_FOLDER_NAME = "bert-base-uncased-tcga"
# trainer.model.save_pretrained(MODEL_SAVE_FOLDER_NAME)
# trainer.save_model(MODEL_SAVE_FOLDER_NAME)
# trainer.model.config.save_pretrained(MODEL_SAVE_FOLDER_NAME)

In [ ]:
# model = AutoModel.from_pretrained("./gatortron-base-tcga")
# tokenizer = AutoTokenizer.from_pretrained("UFNLP/gatortron-base")
# model.push_to_hub("gatortron-base-tcga")